In [16]:
import requests
import json
import bs4
import numpy as np
import pandas as pd
import os

In [2]:
c2h2zff_url_transfac = "http://jaspar.genereg.net/api/v1/matrix/?class=C2H2+zinc+finger+factors&format=transfac"
c2h2zff_url_json = "http://jaspar.genereg.net/api/v1/matrix/?tf_class=C2H2+zinc+finger+factors"#&format=json"
c2h2zffs = requests.get(c2h2zff_url_json)

In [3]:
json.loads(requests.get("http://jaspar.genereg.net/api/v1/matrix/?tf_class=C2H2+zinc+finger+factors").text)

{'count': 428,
 'next': 'http://jaspar.genereg.net/api/v1/matrix/?page=2&tf_class=C2H2+zinc+finger+factors',
 'previous': None,
 'results': [{'matrix_id': 'MA0267.1',
   'name': 'ACE2',
   'collection': 'CORE',
   'base_id': 'MA0267',
   'version': '1',
   'sequence_logo': 'http://jaspar.genereg.net/static/logos/svg/MA0267.1.svg',
   'url': 'http://jaspar.genereg.net/api/v1/matrix/MA0267.1/'},
  {'matrix_id': 'MA1277.1',
   'name': 'Adof1',
   'collection': 'CORE',
   'base_id': 'MA1277',
   'version': '1',
   'sequence_logo': 'http://jaspar.genereg.net/static/logos/svg/MA1277.1.svg',
   'url': 'http://jaspar.genereg.net/api/v1/matrix/MA1277.1/'},
  {'matrix_id': 'MA0268.1',
   'name': 'ADR1',
   'collection': 'CORE',
   'base_id': 'MA0268',
   'version': '1',
   'sequence_logo': 'http://jaspar.genereg.net/static/logos/svg/MA0268.1.svg',
   'url': 'http://jaspar.genereg.net/api/v1/matrix/MA0268.1/'},
  {'matrix_id': 'MA1160.1',
   'name': 'AT1G14580',
   'collection': 'CORE',
   'base_

In [4]:
json.loads(requests.get('http://jaspar.genereg.net/api/v1/matrix/UN0355.1/').text)

{'comment': 'not supported by literature',
 'pubmed_ids': [],
 'family': [],
 'pfm': {'A': [222.0,
   294.0,
   44.0,
   1068.0,
   35.0,
   23.0,
   40.0,
   1077.0,
   887.0,
   234.0,
   325.0],
  'C': [290.0,
   286.0,
   90.0,
   10.0,
   1013.0,
   1078.0,
   5.0,
   7.0,
   139.0,
   362.0,
   205.0],
  'T': [466.0,
   413.0,
   951.0,
   29.0,
   34.0,
   12.0,
   1049.0,
   26.0,
   79.0,
   454.0,
   475.0],
  'G': [146.0, 131.0, 39.0, 17.0, 42.0, 11.0, 30.0, 14.0, 19.0, 74.0, 119.0]},
 'tax_group': 'plants',
 'matrix_id': 'UN0355.1',
 'sequence_logo': 'http://jaspar.genereg.net/static/logos/svg/UN0355.1.svg',
 'pazar_tf_ids': [],
 'versions_url': 'http://jaspar.genereg.net/api/v1/matrix/UN0355/versions',
 'collection': 'UNVALIDATED',
 'base_id': 'UN0355',
 'class': ['C2H2 zinc finger factors'],
 'tffm': None,
 'tfe_ids': [],
 'name': 'AT3G49930',
 'uniprot_ids': ['Q9SN24'],
 'sites_url': 'http://jaspar.genereg.net/api/v1/sites/UN0355.1',
 'centrality_logp': '-379.729',
 'sou

In [5]:
json.loads(requests.get('http://jaspar.genereg.net/api/v1/matrix/MA0267.1/').text)

{'tffm': None,
 'tfe_ids': [],
 'pubmed_ids': ['19111667'],
 'name': 'ACE2',
 'family': ['Other factors with up to three adjacent zinc fingers'],
 'uniprot_ids': ['P21192'],
 'pfm': {'A': [45.0, 1.0, 0.0, 90.0, 0.0, 0.0, 49.0],
  'C': [28.0, 92.0, 100.0, 10.0, 0.0, 100.0, 20.0],
  'T': [5.0, 1.0, 0.0, 0.0, 0.0, 0.0, 14.0],
  'G': [21.0, 6.0, 0.0, 0.0, 100.0, 0.0, 17.0]},
 'tax_group': 'fungi',
 'sites_url': None,
 'matrix_id': 'MA0267.1',
 'sequence_logo': 'http://jaspar.genereg.net/static/logos/svg/MA0267.1.svg',
 'species': [{'name': 'Saccharomyces cerevisiae', 'tax_id': 4932}],
 'pazar_tf_ids': [],
 'versions_url': 'http://jaspar.genereg.net/api/v1/matrix/MA0267/versions',
 'version': 1,
 'collection': 'CORE',
 'base_id': 'MA0267',
 'type': 'PBM, CSA and/or DIP-chip',
 'class': ['C2H2 zinc finger factors']}

In [8]:
def parse_pages(n_pages):
    """
    n_pages: probably 43 (can be drawn from count)
    """
    lst = []
    for i in range(n_pages):
        print(i)
        url_i = f'http://jaspar.genereg.net/api/v1/matrix/?page={i+1}&tf_class=C2H2+zinc+finger+factors'
        url_i_text = requests.get(url_i).text
        page = json.loads(url_i_text)
        parse_page(page, lst)
    return pd.DataFrame(lst)

def parse_page(page, lst):
    """
    Parses an individual page of results (usually 10)
    page: the page of search results
    lst: where the desired pieces of information are appended
    """
    for entry in page['results']:
        entry_url = entry['url']
        entry_url_json = json.loads(requests.get(entry_url).text)
       
        possible_uniprot_id = entry_url_json['uniprot_ids']
        matrix_id = entry_url_json['matrix_id']
        base_id = entry_url_json['base_id']
        name = entry_url_json['name']
        comment = np.NaN
      
        if 'comment' in entry_url_json.keys():
            comment = entry_url_json['comment']
        if len(possible_uniprot_id) == 1:
            possible_uniprot_id = possible_uniprot_id[0]
        else:
            possible_uniprot_id = np.NaN
        lst.append({'Uniprot_IDs': possible_uniprot_id, 
                    'Name': name,
                    'Base_ID': base_id,
                    'Matrix_ID': matrix_id,
                    'PFM': PFMdict_to_matrix(entry_url_json['pfm']),
                    'Comment': comment})

def PFMdict_to_matrix(PFMdict):
    """
    Turns an individual PFM from the format in which it is stored (dictionary, from json) into a np array
    PFMdict: a dictionary containing the PFM
    """
    DNAletters = ['A', 'C', 'G', 'T']
    return np.array([PFMdict[i] for i in DNAletters])

In [10]:
JASPAR_data = parse_pages(43)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42


In [14]:
JASPAR_data[JASPAR_data['Comment'] == 'not supported by literature']

,Uniprot_IDs,Name,Base_ID,Matrix_ID,PFM,Comment
9,Q9SN24,AT3G49930,UN0355,UN0355.1,"[[222.0, 294.0, 44.0, 1068.0, 35.0, 23.0, 40.0...",not supported by literature
14,Q9C0K0,BCL11B,UN0114,UN0114.1,"[[0.0, 2.0, 0.0, 848.0, 848.0, 1.0, 168.0, 58....",not supported by literature
28,Q9SEZ3,CDF5,UN0360,UN0360.1,"[[1306.0, 826.0, 3821.0, 3802.0, 3820.0, 3820....",not supported by literature
29,A0A090M8Q1,CEG01538.1,UN0275,UN0275.1,"[[69.0, 112.0, 999.0, 0.0, 0.0, 0.0, 111.0, 13...",not supported by literature
33,Q5KBN4,CNI01970,UN0099,UN0099.1,"[[107.0, 25.0, 0.0, 0.0, 0.0, 774.0, 7.0, 165....",not supported by literature
...,...,...,...,...,...,...
420,Q9H4T2,ZSCAN16,UN0244,UN0244.1,"[[26957.0, 43086.0, 7390.0, 33.0, 4.0, 3.0, 2....",not supported by literature
422,Q3MJ62,ZSCAN23,UN0246,UN0246.1,"[[223.0, 91.0, 49.0, 1118.0, 7.0, 34.0, 0.0, 0...",not supported by literature
424,Q96LW9,ZSCAN31,UN0247,UN0247.1,"[[1041.0, 51.0, 6693.0, 3.0, 6722.0, 7143.0, 3...",not supported by literature
426,Q9BUG6,ZSCAN5A,UN0248,UN0248.1,"[[3916.0, 4000.0, 2792.0, 2334.0, 1670.0, 747....",not supported by literature


## merge with UniProt

In [17]:
fp = os.path.join('data', 'Transcription Factors (UniProt).xlsx')
uniprot_zf = pd.read_excel(fp)
uniprot_zf.rename(columns={'Entry': 'Uniprot ID'}, inplace=True)

In [19]:
JASPAR_merged = JASPAR_data.merge(uniprot_zf, left_on='Uniprot_IDs', right_on='Uniprot ID', how='left')

In [22]:
JASPAR_merged[JASPAR_merged['Uniprot ID'].isnull()]

,Uniprot_IDs,Name,Base_ID,Matrix_ID,PFM,Comment,Uniprot ID,Entry name,Status,Protein names,Gene names,Organism,Length,Sequence,Zinc finger
8,Q9LX86,AT3G46070,MA1381,MA1381.1,"[[45.0, 27.0, 78.0, 63.0, 18.0, 48.0, 9.0, 0.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Q9SN24,AT3G49930,UN0355,UN0355.1,"[[222.0, 294.0, 44.0, 1068.0, 35.0, 23.0, 40.0...",not supported by literature,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,A8KA13,BCL6B,MA0731,MA0731.1,"[[0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 280....",Data is from Taipale HTSELEX DBD (2013),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,Q94513,BEAF-32,MA0529,MA0529.2,"[[1772.0, 1612.0, 539.0, 5151.0, 98.0, 26.0, 4...",Motif with less uninformative columns,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,Q01295,br,MA0010,MA0010.1,"[[3.0, 1.0, 5.0, 7.0, 3.0, 6.0, 4.0, 7.0, 1.0,...",Different splice forms affect DNA binding: fou...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,Q5HY98,ZNF766,UN0223,UN0223.1,"[[2425.0, 2941.0, 6635.0, 7378.0, 51.0, 7935.0...",Matches the ChIP-seq derived motif,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401,Q6IQ21,ZNF770,UN0225,UN0225.1,"[[1127.0, 1464.0, 292.0, 3968.0, 179.0, 101.0,...",No other support,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
404,Q8N8C0,ZNF781,UN0228,UN0228.1,"[[1117.0, 367.0, 3419.0, 67.0, 4437.0, 97.0, 3...",TF without reported motif. More evidences were...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
405,Q8NCA9,ZNF784,UN0229,UN0229.1,"[[1757.0, 1762.0, 6529.0, 26.0, 71.0, 206.0, 6...",not supported by literature,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
